In [1]:
import numpy as np
seed=7
np.random.seed(seed)

In [2]:
import tensorflow as tp
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import pandas as pd
from keras import backend as K
# from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
#checking if GPU is used 
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [4]:
dataset = pd.read_csv("../../data/1.csv", delimiter=",")
#dataset

In [5]:
def mean_imputation(dataset): 
    """"""
    imputed_dataset=dataset.fillna(dataset.mean())
    return imputed_dataset

In [6]:
dataset=mean_imputation(dataset)
#dataset

In [7]:
#to _numpy() works for 0.24.2 + version of pandas
pd.__version__

'0.24.2'

In [8]:
#dataset_np_array=dataset.to_numpy()
dataset_np_array=dataset.values
dataset_np_array

array([[ 18. ,   8. , 307. , ...,  12. ,  70. ,   1. ],
       [ 15. ,   8. , 350. , ...,  11.5,  70. ,   1. ],
       [ 18. ,   8. , 318. , ...,  11. ,  70. ,   1. ],
       ...,
       [ 32. ,   4. , 135. , ...,  11.6,  82. ,   1. ],
       [ 28. ,   4. , 120. , ...,  18.6,  82. ,   1. ],
       [ 31. ,   4. , 119. , ...,  19.4,  82. ,   1. ]])

In [10]:
#split into input (X) and output (Y)
def splitXY(dataset): 
    """
    Takes numpy array as input and converts first column into Y and rest into X
    """
    Y=dataset[:,0]
    X=dataset[:,1:]
    return X,Y

X,Y= splitXY(dataset_np_array)

In [34]:
m,n=X.shape
X1=X[:,0:7]
X1

array([[  8. , 307. , 130. , ...,  12. ,  70. ,   1. ],
       [  8. , 350. , 165. , ...,  11.5,  70. ,   1. ],
       [  8. , 318. , 150. , ...,  11. ,  70. ,   1. ],
       ...,
       [  4. , 135. ,  84. , ...,  11.6,  82. ,   1. ],
       [  4. , 120. ,  79. , ...,  18.6,  82. ,   1. ],
       [  4. , 119. ,  82. , ...,  19.4,  82. ,   1. ]])

In [35]:
def forward_sel(X,Y):
    m,n=X.shape
    for colno in range(0,n):
        X=X[:,0:colno+1]
        #run model from here..
    return None 

In [11]:
def r_squared(y_true, y_pred):
    sse=tp.reduce_sum(tp.square(tp.subtract(y_pred,y_true)))
   
    
    #using sst as y_true-y_mean 's squared mean
    y_mean= tp.reduce_mean(y_true)
    sst=tp.reduce_sum(tp.square(tp.subtract(y_true,y_mean)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    
    """
    m=tp.to_float(tp.size(y_true))
    y_true_sum_sq=(tp.square(tp.reduce_sum(y_true)))
    y_sq_mean=tp.divide(y_true_sum_sq,m)
    sst=tp.subtract(tp.tensordot(y_true,y_true,0),m*tp.square(tp.reduce_mean(y_true)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    """
    return r_square

In [25]:
def adj_r_squared(y_true, y_pred):
    sse=tp.reduce_sum(tp.square(tp.subtract(y_pred,y_true)))
    #using sst as y_true-y_mean 's squared mean
    y_mean= tp.reduce_mean(y_true)
    sst=tp.reduce_sum(tp.square(tp.subtract(y_true,y_mean)))
    r_square=tp.subtract(float(1),tp.divide(sse,sst))
    #n=model.count_params()#this part was working before but now is not..how to get model here though?
    #n=sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])
    n=count
    m=tp.to_float(tp.size(y_true))
    rdf=tp.divide(tp.subtract(m,float(1)),tp.subtract(m,n))
    r_adj_square=tp.subtract(float(1),tp.tensordot(rdf,tp.subtract(float(1),r_square),0))
    return r_adj_square

In [13]:
def perceptronModel(X_dim=7,activation_name='sigmoid'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(1, input_dim=X_dim,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    global count
    count=model.count_params()
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    sgd=optimizers.SGD(lr=0.1, momentum=0.02, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [20]:
def neuralNetwork3LModel(X_dim,activation_name='sigmoid'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(X_dim, input_dim=X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    global count
    count=model.count_params()-X_dim+1
    sgd=optimizers.SGD(lr=0.1, momentum=0.02, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [21]:
def neuralNetwork5LModel(X_dim,activation_name='sigmoid'):
    """Takes number of X features and activation name as input and outputs a keras model for perceptron"""
    model = Sequential()
    model.add(Dense(X_dim, input_dim=X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(X_dim,activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    model.add(Dense(1,activation='linear',kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    #model.add(Dense(1,kernel_initializer='normal'))
    
    #model.add(Dense(1, input_dim=X_dim, activation=activation_name,kernel_initializer=keras.initializers.RandomNormal(stddev=1),bias_initializer=keras.initializers.RandomNormal(stddev=1)))
    # Compile model
    global count
    count=model.count_params()-2*X_dim +1
    sgd=optimizers.SGD(lr=0.1, momentum=0.02, decay=0.0, nesterov=False)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse',r_squared,adj_r_squared])

    
    return model

In [22]:
(X_size,X_dim)=X.shape
X_dim

7

In [24]:

#model =perceptronModel(X_dim)
#model =neuralNetwork3LModel(X_dim)
model =neuralNetwork5LModel(X_dim)

model.summary()

# Fit the model
model.fit(X, Y, epochs=500, batch_size=30)

# evaluate the model
# evaluate model with standardized dataset
#estimator = KerasRegressor(build_fn=perceptronModel, epochs=100, batch_size=5, verbose=0)

# kfold = KFold(n_splits=10, random_state=seed)
# results = cross_val_score(estimator, X, Y, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 56        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 8         
Total params: 176
Trainable params: 176
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
406/406 [==============================] - 1s 1ms/step - loss: 665.0863 - mean_squared_error: 665.0863 - r_squared: -11.0056 - adj_r_squared: 3.5797
Epoch 2/500
406/406 [==============================] - 0s 209us/step - loss: 658.2472 - mean_squared_error: 658.247

406/406 [==============================] - 0s 120us/step - loss: 387.0171 - mean_squared_error: 387.0171 - r_squared: -5.8224 - adj_r_squared: 2.4585
Epoch 47/500
406/406 [==============================] - 0s 121us/step - loss: 382.3153 - mean_squared_error: 382.3153 - r_squared: -5.7279 - adj_r_squared: 2.4438
Epoch 48/500
406/406 [==============================] - 0s 123us/step - loss: 377.7239 - mean_squared_error: 377.7239 - r_squared: -5.6229 - adj_r_squared: 2.4113
Epoch 49/500
406/406 [==============================] - 0s 112us/step - loss: 373.2768 - mean_squared_error: 373.2768 - r_squared: -5.5811 - adj_r_squared: 2.3990
Epoch 50/500
406/406 [==============================] - 0s 128us/step - loss: 368.9633 - mean_squared_error: 368.9633 - r_squared: -5.5355 - adj_r_squared: 2.3956
Epoch 51/500
406/406 [==============================] - 0s 122us/step - loss: 364.8353 - mean_squared_error: 364.8353 - r_squared: -5.7067 - adj_r_squared: 2.4365
Epoch 52/500
406/406 [=============

406/406 [==============================] - 0s 118us/step - loss: 252.6389 - mean_squared_error: 252.6389 - r_squared: -3.4363 - adj_r_squared: 1.9454
Epoch 97/500
406/406 [==============================] - 0s 148us/step - loss: 250.8171 - mean_squared_error: 250.8171 - r_squared: -3.3645 - adj_r_squared: 1.9355
Epoch 98/500
406/406 [==============================] - 0s 118us/step - loss: 248.9936 - mean_squared_error: 248.9936 - r_squared: -3.4146 - adj_r_squared: 1.9452
Epoch 99/500
406/406 [==============================] - 0s 114us/step - loss: 247.1824 - mean_squared_error: 247.1824 - r_squared: -3.5637 - adj_r_squared: 1.9770
Epoch 100/500
406/406 [==============================] - 0s 113us/step - loss: 245.3255 - mean_squared_error: 245.3255 - r_squared: -3.3537 - adj_r_squared: 1.9279
Epoch 101/500
406/406 [==============================] - 0s 153us/step - loss: 243.5450 - mean_squared_error: 243.5450 - r_squared: -3.2535 - adj_r_squared: 1.9092
Epoch 102/500
406/406 [==========

406/406 [==============================] - 0s 134us/step - loss: 151.5514 - mean_squared_error: 151.5514 - r_squared: -1.7736 - adj_r_squared: 1.5777
Epoch 147/500
406/406 [==============================] - 0s 143us/step - loss: 150.1830 - mean_squared_error: 150.1830 - r_squared: -1.6833 - adj_r_squared: 1.5758
Epoch 148/500
406/406 [==============================] - 0s 113us/step - loss: 148.8318 - mean_squared_error: 148.8318 - r_squared: -1.7036 - adj_r_squared: 1.5759
Epoch 149/500
406/406 [==============================] - 0s 130us/step - loss: 147.5147 - mean_squared_error: 147.5147 - r_squared: -1.5721 - adj_r_squared: 1.5477
Epoch 150/500
406/406 [==============================] - 0s 117us/step - loss: 146.2025 - mean_squared_error: 146.2025 - r_squared: -1.5472 - adj_r_squared: 1.5459
Epoch 151/500
406/406 [==============================] - 0s 118us/step - loss: 144.9304 - mean_squared_error: 144.9304 - r_squared: -1.6970 - adj_r_squared: 1.5660
Epoch 152/500
406/406 [=======

406/406 [==============================] - 0s 123us/step - loss: 101.6728 - mean_squared_error: 101.6728 - r_squared: -0.7827 - adj_r_squared: 1.3811
Epoch 197/500
406/406 [==============================] - 0s 118us/step - loss: 100.9427 - mean_squared_error: 100.9427 - r_squared: -0.7923 - adj_r_squared: 1.3789
Epoch 198/500
406/406 [==============================] - 0s 111us/step - loss: 100.2622 - mean_squared_error: 100.2622 - r_squared: -0.7455 - adj_r_squared: 1.3725
Epoch 199/500
406/406 [==============================] - 0s 122us/step - loss: 99.5596 - mean_squared_error: 99.5596 - r_squared: -0.7079 - adj_r_squared: 1.3626
Epoch 200/500
406/406 [==============================] - 0s 112us/step - loss: 98.8842 - mean_squared_error: 98.8842 - r_squared: -0.7613 - adj_r_squared: 1.3734
Epoch 201/500
406/406 [==============================] - 0s 118us/step - loss: 98.2149 - mean_squared_error: 98.2149 - r_squared: -0.7001 - adj_r_squared: 1.3609
Epoch 202/500
406/406 [=============

...

406/406 [==============================] - 0s 118us/step - loss: 62.5142 - mean_squared_error: 62.5142 - r_squared: -0.0698 - adj_r_squared: 1.2287
Epoch 297/500
406/406 [==============================] - 0s 119us/step - loss: 62.4058 - mean_squared_error: 62.4058 - r_squared: -0.0787 - adj_r_squared: 1.2303
Epoch 298/500
406/406 [==============================] - 0s 122us/step - loss: 62.2922 - mean_squared_error: 62.2922 - r_squared: -0.0725 - adj_r_squared: 1.2290
Epoch 299/500
406/406 [==============================] - 0s 113us/step - loss: 62.1772 - mean_squared_error: 62.1772 - r_squared: -0.0570 - adj_r_squared: 1.2256
Epoch 300/500
406/406 [==============================] - 0s 130us/step - loss: 62.0845 - mean_squared_error: 62.0845 - r_squared: -0.0658 - adj_r_squared: 1.2268
Epoch 301/500
406/406 [==============================] - 0s 112us/step - loss: 61.9697 - mean_squared_error: 61.9697 - r_squared: -0.0811 - adj_r_squared: 1.2295
Epoch 302/500
406/406 [===================

406/406 [==============================] - 0s 119us/step - loss: 59.9749 - mean_squared_error: 59.9749 - r_squared: -0.0359 - adj_r_squared: 1.2210
Epoch 347/500
406/406 [==============================] - 0s 129us/step - loss: 59.9620 - mean_squared_error: 59.9620 - r_squared: -0.0734 - adj_r_squared: 1.2287
Epoch 348/500
406/406 [==============================] - 0s 181us/step - loss: 59.9486 - mean_squared_error: 59.9486 - r_squared: -0.0273 - adj_r_squared: 1.2193
Epoch 349/500
406/406 [==============================] - 0s 153us/step - loss: 59.9360 - mean_squared_error: 59.9360 - r_squared: -0.0352 - adj_r_squared: 1.2211
Epoch 350/500
406/406 [==============================] - 0s 144us/step - loss: 59.9273 - mean_squared_error: 59.9273 - r_squared: -0.0375 - adj_r_squared: 1.2216
Epoch 351/500
406/406 [==============================] - 0s 112us/step - loss: 59.9141 - mean_squared_error: 59.9141 - r_squared: -0.0362 - adj_r_squared: 1.2211
Epoch 352/500
406/406 [===================

406/406 [==============================] - 0s 119us/step - loss: 59.7500 - mean_squared_error: 59.7500 - r_squared: -0.0306 - adj_r_squared: 1.2199
Epoch 397/500
406/406 [==============================] - 0s 122us/step - loss: 59.7471 - mean_squared_error: 59.7471 - r_squared: -0.0340 - adj_r_squared: 1.2209
Epoch 398/500
406/406 [==============================] - 0s 119us/step - loss: 59.7461 - mean_squared_error: 59.7461 - r_squared: -0.0135 - adj_r_squared: 1.2164
Epoch 399/500
406/406 [==============================] - 0s 118us/step - loss: 59.7494 - mean_squared_error: 59.7494 - r_squared: -0.0349 - adj_r_squared: 1.2210
Epoch 400/500
406/406 [==============================] - 0s 118us/step - loss: 59.7449 - mean_squared_error: 59.7449 - r_squared: -0.0236 - adj_r_squared: 1.2184
Epoch 401/500
406/406 [==============================] - 0s 171us/step - loss: 59.7443 - mean_squared_error: 59.7443 - r_squared: -0.0488 - adj_r_squared: 1.2235
Epoch 402/500
406/406 [===================

406/406 [==============================] - 0s 140us/step - loss: 59.7361 - mean_squared_error: 59.7361 - r_squared: -0.0292 - adj_r_squared: 1.2197
Epoch 447/500
406/406 [==============================] - 0s 118us/step - loss: 59.7372 - mean_squared_error: 59.7372 - r_squared: -0.0444 - adj_r_squared: 1.2223
Epoch 448/500
406/406 [==============================] - 0s 129us/step - loss: 59.7380 - mean_squared_error: 59.7380 - r_squared: -0.0365 - adj_r_squared: 1.2212
Epoch 449/500
406/406 [==============================] - 0s 118us/step - loss: 59.7361 - mean_squared_error: 59.7361 - r_squared: -0.0247 - adj_r_squared: 1.2187
Epoch 450/500
406/406 [==============================] - 0s 112us/step - loss: 59.7363 - mean_squared_error: 59.7363 - r_squared: -0.0109 - adj_r_squared: 1.2158
Epoch 451/500
406/406 [==============================] - 0s 134us/step - loss: 59.7437 - mean_squared_error: 59.7437 - r_squared: -0.0497 - adj_r_squared: 1.2229
Epoch 452/500
406/406 [===================

406/406 [==============================] - 0s 168us/step - loss: 59.7360 - mean_squared_error: 59.7360 - r_squared: -0.0246 - adj_r_squared: 1.2188
Epoch 497/500
406/406 [==============================] - 0s 177us/step - loss: 59.7395 - mean_squared_error: 59.7395 - r_squared: -0.0640 - adj_r_squared: 1.2262
Epoch 498/500
406/406 [==============================] - 0s 123us/step - loss: 59.7388 - mean_squared_error: 59.7388 - r_squared: -0.0265 - adj_r_squared: 1.2191
Epoch 499/500
406/406 [==============================] - 0s 119us/step - loss: 59.7363 - mean_squared_error: 59.7363 - r_squared: -0.0288 - adj_r_squared: 1.2183
Epoch 500/500
406/406 [==============================] - 0s 118us/step - loss: 59.7386 - mean_squared_error: 59.7386 - r_squared: -0.0710 - adj_r_squared: 1.2287


In [ ]:
#the model is not learning
#what can be wrong? 
# basic mistake can be how X and Y are designed, but this is unlikely.
#somehow 